In [1]:
import numpy as np
import time
import sys
import os
sys.path.append('C:\\git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('C:\\git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('C:\git_repos\lab_utilities\IO_3001_USB_acquisition')
sys.path.append('C:\\Anaconda3\\envs\\py27\\Scripts')
from daq import daqDevice
import daqh
from prm_util import nps
import h5py
%pylab

import prm_util as cu

from Tkinter import Tk
from tkFileDialog import askopenfilename

acqmode=daqh.DaamInfinitePost

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def get_data(nchan=4,freq=10,nseconds=5,comment='None',alerts=[58,59,60,118,119,120,178,179,180,238,239,240,298,299,300]):
    """
    function to simply aquire nchan a/d channels at rate freq
    for nseconds seconds
    """
    
    #outdata=np.zeros([nchan,nscans],dtype=float)
    dev=daqDevice('DaqBoard3001USB')
    gains=[]
    flags=[]
    chans=[]
    if nchan > 8:
        uchan=nchan-8
        for i in range(8):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
        for i in range(uchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(256+i)   #HERE is the famous fix where DaqX refs upper level dif channels!
    elif nchan<9:      
        for i in range(nchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
    acqmode = daqh.DaamNShot
    dev.AdcSetAcq(acqmode, postTrigCount = nseconds*freq)
    dev.AdcSetScan(chans,gains,flags)
    dev.AdcSetFreq(freq)
    #use the driver buffer here user buffer was very limited (the way I tried anyway) 
    transMask = daqh.DatmUpdateBlock|daqh.DatmCycleOn|daqh.DatmDriverBuf

    buf=dev.AdcTransferSetBuffer(transMask, np.uint(nseconds*freq*nchan))
    #bufMask is for transferring the buffer
    bufMask = daqh.DabtmOldest | daqh.DabtmRetAvail

    timestart = (time.time())
    timenotify = timestart + 5

    dev.SetTriggerEvent(daqh.DatsImmediate,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStartEvent)
    dev.SetTriggerEvent(daqh.DatsScanCount,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStopEvent)
    dev.AdcTransferStart()
    dev.AdcArm()
    
    while True:
        alertscopy=alerts[:]
        timenotify = checkAlerts(timenotify, timestart, alerts,alertscopy)        
        
        stat = dev.AdcTransferGetStat()
        active = stat['active']
        if not (active & daqh.DaafAcqActive):
            break
    dev.AdcDisarm()
    outdata,ret=dev.AdcTransferBufData(nseconds*freq, nchan,bufMask)
    
    outdata=np.array(outdata,dtype=float)
    outdata=(outdata-2**15)*20./2**16
    outdata=np.transpose(np.reshape(outdata,[nseconds*freq,nchan]))
    print "Finished collecting data\n-"
    dev.Close()    
    return outdata

In [4]:
def checkAlerts(timeupdate,timestart, alerts, alertscopy, updateincrement=5):
    timecheck = (time.time())
    if timecheck > timeupdate:
        print "Time: " + str(int(timecheck - timestart))
        timeupdate += updateincrement
    for alert in alerts:
        if (timecheck - timestart)%60 > alert:
            print "---- " + str(int(timecheck - timestart)) + " SECONDS ----"
            alerts.remove(alert)
    if 2<(timecheck -timestart%60)<3:
        alerts = alertscopy[:]
        
        
    return timeupdate

In [5]:
def get_date_filename(labelstring=''):
    #version for testing adds an optional string to identify tests
    now=time.localtime()[0:6]
    dirfmt = "C:\\Interferometer_tests\\data\\%4d_%02d_%02d"
    dirname = dirfmt % now[0:3]
    filefmt = "%02d_%02d_%02d.h5"
    filename= labelstring+filefmt % now[3:6]
    ffilename=os.path.join(dirname,filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    return(ffilename)

In [6]:
def zplot(d_dict_list,minfreq=1,chan=0):
    """
    calculate asd's and overplot all from list of data dictionaries, assumed to have meaningful
    keynames for legend
    """
    figure()
    for d_dict in d_dict_list:
        if len(d_dict['data'])>2:
            z=cu.nps(d_dict['data'],d_dict['samprate'],minfreq=minfreq)
        else:
            z=cu.nps(d_dict['data'][chan],d_dict['samprate'],minfreq=minfreq)
        plot(z[0],np.sqrt(z[1]),label=d_dict['label'])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    xscale('log')
    yscale('log')
    legend()
    title('Comparison spectra')
    legend()

In [7]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test(samprate=10000,nseconds=10,labelstring='test',minfreq=1):
    s=[0]  #active channels, intervening ones are terminated to increase isolation
    d=get_data(nchan=1,freq=samprate,nseconds=nseconds)
    outdata=squeeze(d[s,:])
    fname=get_date_filename(labelstring)
    with h5py.File(fname, mode='w') as hdf_file:
        hdf_file.create_dataset('data', data=outdata)
        hdf_file.create_dataset('samprate', data=[samprate])
        hdf_file.create_dataset('label', data=[labelstring])
        hdf_file.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [8]:
def allplot(d,closef=True,norm=True):
    '''
    set closef to False to overplot on previous figures
    '''
    minfreq=.2
    sr=np.float(d['samprate'])
    x=arange(len(d['data'][0]))/sr
    i=d['data'][0]-np.mean(d['data'][0])
    q=d['data'][1]-np.mean(d['data'][1])
    if norm:
        i=i/np.std(i)
        q=q/np.std(q)
    z=i-mean(i) + 1j*(q-mean(q))
    phi=np.unwrap(np.angle(z))
    if closef:
        close(1)
        close(2)
        close(3)
    figure(1)
    plot(x,phi,label=d['label'])
    xlabel('Time [seconds]')
    ylabel('Phase [Radians]')
    title(d['label'] + ' Phase')
    legend()
    figure(2)
    plot(i,q,'.')
    xlabel('I')
    ylabel('Q')
    title(d['label']+ ' Complex plane')
    legend()
    
    figure(3)
    plot(x,i,label='I '+d['label'])
    plot(x,q,label='Q '+d['label'])
    legend()
    xlabel('Time [Seconds]')
    ylabel('Signal, [au]')
    title(d['label'])
    legend()
    
    figure(4)
    zphi=cu.nps(phi,d['samprate'])
    plot(zphi[0],np.sqrt(zphi[1]),label='Phase  '+d['label'])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    xscale('log')
    yscale('log')
    title('Phase spectrum')
    legend()

In [9]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test2(samprate=10000,nseconds=10,labelstring='test',minfreq=1):
    s=[0,2]  #active channels, intervening ones are terminated to increase isolation
    d=get_data(nchan=4,freq=samprate,nseconds=nseconds)
    outdata=squeeze(d[s,:])
    fname=get_date_filename(labelstring)
    with h5py.File(fname, mode='w') as hdf_file:
        hdf_file.create_dataset('data', data=outdata)
        hdf_file.create_dataset('samprate', data=[samprate])
        hdf_file.create_dataset('label', data=[labelstring])
        hdf_file.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [69]:
def import_data(filename=None):
    '''
    function to read in previously stored data and put in format of freshly read data dictionary
    '''
    
    if filename==None:
        Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        filename = askopenfilename(initialdir='C:\\Interferometer_tests\\io_3001_usb_data')
    hf=h5py.File(filename)
    outdata=reshape(hf['data'][:],[2,-1])
    samprate=hf['samprate'][0]
    labelstring=hf['label'][0]
    hf.close()
    return {'data':outdata,'samprate':samprate,'label':labelstring}

In [10]:
def demod(indata):
    #function to extract I,Q from data, sync
    dd=indata['data'][1,4:] - indata['data'][1,:-4]
    signal=indata['data'][0,4:]
    x=arange(len(dd))
    d_edges=x[where(dd > 2.)]
    u_edges=x[where(dd < -2.)]
    #trim these lists to avoid partials at the end, and force first edge to be rising 
    #last edge to be falling one
    u_edges=u_edges[1:-1]
    d_edges=d_edges[where(d_edges > u_edges[0])]
    d_edges=d_edges[where(d_edges < u_edges[-1])]
    phase1=[]
    phase2=[]
    print(len(d_edges),len(u_edges))
    for u,d,u2 in zip(u_edges[:-1],d_edges,u_edges[1:]):
        if(((d-u)>5) and ((d-u) <100) and ((u2-d>5)) and ((u2-d) <100)):
            phase1.append(np.mean(signal[u:d]))
            phase2.append(np.mean(signal[d:u2]))
    i=np.array(phase1)
    q=np.array(phase2)
    z=i-mean(i) + 1j*(q-mean(q))
    phi=np.unwrap(np.angle(z))
    return i,q,phi
    #return x,u_edges,d_edges,dd

In [58]:
d4=get_test2(nseconds=20,samprate=20000,labelstring='DUT_smf25km_tap')

Time: 5
Time: 10
Time: 15
Time: 20
Finished collecting data
-


In [69]:
allplot(d1)

In [65]:
zplot([d1],minfreq=.1,chan=0)

In [77]:
figure()
plot(d4['data'][0])
plot(d4['data'][1])

In [70]:
d=import_data()

In [71]:
allplot(d)
    

Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 180, in __draw_idle_agg
    FigureCanvasAgg.draw(self)
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\backends\backend_agg.py", line 474, in draw
    self.figure.draw(self.renderer)
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\artist.py", line 62, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\figure.py", line 1159, in draw
    func(*args)
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\artist.py", line 62, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\axes\_base.py", line 2319, in draw
    a.draw(renderer)
  File "C:\Program Files\Anaconda3\envs\py27\lib\site-packages\matplotlib\artist.py", l

In [68]:
d['label']

'test'

In [66]:
df

<HDF5 file "test17_24_07.h5" (mode r+)>